In [2]:
import cv2

In [3]:
import numpy as np

In [8]:
from keras.preprocessing import image

In [5]:
from keras.applications import vgg16

In [6]:
img_rows, img_cols=244,244

In [11]:
model=vgg16.VGG16(weights='imagenet', include_top = False,
                 input_shape = (244,244,3))

In [11]:
model.layers

In [10]:
for (i,layer) in enumerate(model.layers):
    print(str(i)+"  "+model.layers[0].__class__.__name__+"  "+str(model.layers[i].trainable))

0  InputLayer  False
1  InputLayer  True
2  InputLayer  True
3  InputLayer  True
4  InputLayer  True
5  InputLayer  True
6  InputLayer  True
7  InputLayer  True
8  InputLayer  True
9  InputLayer  True
10  InputLayer  True
11  InputLayer  True
12  InputLayer  True
13  InputLayer  True
14  InputLayer  True
15  InputLayer  True
16  InputLayer  True
17  InputLayer  True
18  InputLayer  True


In [11]:
model.layers[0].input

<tf.Tensor 'input_3:0' shape=(?, 244, 244, 3) dtype=float32>

In [12]:
for layer in model.layers:
    layer.trainable = False

In [13]:
def addTopModel(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 3

FC_Head = addTopModel(model, num_classes)

new_model = Model(inputs = model.input, outputs = FC_Head)

print(new_model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 244, 244, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 244, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 244, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 122, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 122, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 122, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 61, 61, 128)       0   

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'C:/Users/DELL/Pictures/Camera Roll/train/'
validation_data_dir = 'C:/Users/DELL/Pictures/Camera Roll/validation/'

# Let's use some data augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 32
val_batchsize = 16
img_rows=224
img_cols=224
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("facedetect.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
new_model.compile(loss = 'binary_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history = new_model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

new_model.save("facedetect.h5")


In [ ]:
from keras.models import load_model

model = load_model('facedetect.h5')

In [ ]:
train_generator.class_indices

In [ ]:
import cv2
import numpy as np


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):

    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (224, 224))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    image, face = face_detector(frame)

    face=np.array(face)
    face=np.expand_dims(face,axis=0)
    if face.shape==(1,0):
        cv2.putText(image,"I don't know", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        cv2.imshow('Face Recognition',image)
    else:
        result=model.predict(face)
        print(result)
        if result[0][0] == 1.0:
            cv2.putText(image,"Narendra", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition',image)
        elif result[0][0] == 0.0:
            cv2.putText(image,"Sunil", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition',image)
        else:
            cv2.putText(image,"Not recognized", (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face Recognition',image)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()